# Tratamento de dados de um DataFrame de Animes da Crunchyroll

## Importação das bibliotecas e carregamento do DataFrame

In [ ]:
import pandas as pd

# Carregamento dos dados
caminho_arquivo = 'animes.csv'
dados_animes = pd.read_csv(caminho_arquivo)

## Renomeando colunas

In [ ]:
# Renomeando colunas para Português
renomear_colunas = {
    'rate': 'Avaliação',
    'votes': 'Votos Totais',
    'episodes': 'Número de Episódios'
}
dados_animes.rename(columns=renomear_colunas, inplace=True)

## Cálculos estatísticos

In [ ]:
# Adicionando indicadores estatísticos
colunas_genero = [col for col in dados_animes.columns if 'genre_' in col]
anime_popular_por_genero = {}

for genero in colunas_genero:
    dados_genero = dados_animes[dados_animes[genero] == 1]
    if not dados_genero.empty:
        mais_popular = dados_genero.loc[dados_genero['Votos Totais'].idxmax()]
        anime_popular_por_genero[genero] = mais_popular['anime']
        dados_animes.loc[dados_animes['anime'] == mais_popular['anime'], f'{genero}_Anime Mais Popular'] = mais_popular['anime']
        dados_animes.loc[dados_animes['anime'] == mais_popular['anime'], f'{genero}_Gênero Mais Popular'] = genero
        dados_animes[f'{genero}_Média Avaliações'] = dados_genero['Avaliação'].mean()
        dados_animes[f'{genero}_Total Votos Gênero'] = dados_genero['Votos Totais'].sum()
        dados_animes[f'{genero}_Contagem Animes'] = dados_genero.shape[0]
        dados_animes[f'{genero}_Média Episódios'] = dados_genero['Número de Episódios'].mean()

# Identificando animes altamente avaliados
dados_animes['Animes Populares'] = dados_animes.apply(lambda x: x['anime'] if x['Avaliação'] >= 4.5 else '', axis=1)


## Reorganizando os dados

In [ ]:
# Preparando as colunas para reorganização
colunas_nucleo = ['anime', 'anime_url', 'anime_img', 'Número de Episódios', 'Votos Totais', 'Animes Populares', 'Avaliação']
colunas_estatisticas = [col for col in dados_animes.columns if any(palavra_chave in col for palavra_chave in ['Média Avaliações', 'Total Votos Gênero', 'Contagem Animes', 'Média Episódios'])]
colunas_popularidade_genero = [col for col in dados_animes.columns if 'Mais Popular' in col]

# Reorganizando colunas
dados_animes = dados_animes[colunas_nucleo + colunas_popularidade_genero + colunas_estatisticas]

## Exportando os dados tratados para um novo CSV

In [ ]:
# Salvando o dataframe processado
caminho_arquivo_processado = 'Animes_ETL_Final.csv'  # Substitua pelo caminho desejado
dados_animes.to_csv(caminho_arquivo_processado, index=False)

print(f"Arquivo processado salvo em: {caminho_arquivo_processado}")